# INTRODUCTION

## The project

Our project focuses on the Minihack environment called HideNSeek. This environment consists of a labyrinth with trees and clouds. The agent's goal is to reach the stairs without being killed by the monster. There are different versions of this environment, but our project primarily concentrates on the Mapped version, where the environment is fully observable, and the standard version of HideNSeek, where the environment is partially observable and revealed to the agent as it explores the map. Our project is structured around two distinct approaches: a knowledge base and the A* pathfinding algorithm. Initially, both techniques were explored in parallel. For both, we started with a simplified version of the problem, beginning with a fully observable environment. Initially, the map only contained trees, and later clouds and monsters were included.

Subsequently, we integrated the two strategies in different ways, which will be detailed later. Finally, we transitioned to the partially observable version of the problem, where we once again implemented a strategy that combined both techniques.

## Monsters

Different monsters in Nethack have different characteristics:
- Some monsters are slower than the agent, thus they occasionally skip turns;
- Many of the monsters have (powerful) ranged attacks;
- Some monsters have an ability called *infravision*, which means that they can locate the agent no matter the distance or obstacles;

Even though the agent has a leather armor, enabling him to endure hits, many of the monsters in the environment are too powerful to confront with an empty inventory.
As a consequence, we decided to focus on a subset of them, considering only two monsters: the Naga and the Giant Humanoid. The 'Hide and Seek' environment's DES file has been appropriately modified to include only these two creatures.
Both the Naga and the Giant can be lethal both up close and from a distance, but they have their respective strengths and weaknesses. The fact that these two monsters are quite different from each other, allowed us to test our approach effectively on a manageable scale. 
The Naga is a serpent-like creature known for its agility and speed. It is a very fast monster, and its long-range attack is more potent, making it a significant threat when engaged from a distance. 
The Giant is a large humanoid creature, known for its brute strength and durability. It is slower but inflicts substantial damage when in close proximity. This makes it particularly dangerous if the player gets too close. 


## Stats
The empirical evaluation is ran over 500 episodes, with a maximum number of steps of 30, using the default Minihack Reward Manager, which yields a positive score of 1 for a successful episode, and 0 otherwise. We acquired statistics over different monster pools:
- The *full* HideNSeek pool (**Giant, Naga, Titan, Dragon, Ettin, Minotaur, Lich, Ogre, Troll**);
- **Giant** only;
- **Naga** only.

We plotted the number of successful episodes (win/losses), the number of deaths caused by each monster (for the full pool), and the execution time. 

# THE FULLY OBSERVABLE ENVIRONMENT

# KNOWLEDGE BASE APPROACH

In [ ]:
import gym
import time
import matplotlib.pyplot as plt
from pyswip import Prolog
import IPython.display as display
from project_utils import *


## Basic Movement

The first thing we did, after setting up the environment and finding out a way to formulate and update perceptions from the game into the knowledge base, was to design and test the basic movement towards the goal. Here's a test of the agent's movement in a fully mapped environment, with just trees.

In [ ]:

ENV = gym.make("MiniHack-Navigation-Custom-v0",des_file="dat/kb_movement.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))

evaluate(num_ep = 1, max_steps = 30, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "slow", show = True)

## Loops
Since the project's bigger picture would have included a heuristic pathfinding solution, we kept the reasoning "step-by-step" in the Knowledge Base version, without using lists to build a path before moving. For this reason, in this version of the system, loops might occur: it can be the case that the optimal path is blocked by trees or/and walls, and the agent walks back and forth over the same two cells even if these moves lead to a dead end. In the next map we can see an example of this behaviour.

In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0",des_file="dat/kb_loop.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 1, max_steps = 6, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "slow", show = True)

## Loop fix
To avoid the occurrence of loops, we "discouraged" the agent from walking over already-walked cells. 

In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0",des_file="dat/kb_loop.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 1, max_steps = 30, kb_path = 'kbs/project_kb.pl', env = ENV, speed = "slow", show = True)

## Introducing monsters
Now it's time to bring the monsters in.
In this version, monsters are not considered by the agent at all: he just rushes towards the stairs, ignoring them. This didn't lead to horrible results though, because being exposed for a long time or avoiding close contact with the enemy can be counterproductive.
Though not catastrophic, this strategy isn't enough, especially against the **Naga**. She rocks a powerful fire ranged attack, whilst having the same speed as the agent. Since enemies can attack us at range from basically *any* distance, performing intricated movement strategies would both put us at risk of exceeding the maximum number of steps and give the monster more attempts to hit us at range. 

In [ ]:
ENV = gym.make("MiniHack-HideNSeek-Mapped-v0",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 3, max_steps = 30, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "slow", show = True)

## Evaluation 
Now we look at some evaluation, with different monster pools:

In [ ]:
ENV = gym.make("MiniHack-HideNSeek-Mapped-v0",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH
#REMOVE IMPOSSIBLE TO PERFORM ANY ACTION

In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_N.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH
#REMOVE IMPOSSIBLE TO PERFORM ANY ACTION

In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_H.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/looping_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH
#REMOVE IMPOSSIBLE TO PERFORM ANY ACTION

## Dealing with monsters
 One useful thing, though, is that ranged attacks seem to have a *lower* chance to connect if the line of fire is not exactly *perpendicular* (or *diagonal*). Thus, we made the agent less prone to walk in those cells, as well as the cells strictly adjacent to the monster. Also, the agent keeps tracks of the last enemy position, when it disappears into clouds.


In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_NH.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 3, max_steps = 30, kb_path = 'kbs/project_kb.pl', env = ENV, speed = "slow", show = True)

## Evaluation II
Again, some evaluation on the different monster pools. The Knowledge Base tweaks brought an improvement in all of the three monster pools performance.

In [ ]:
ENV = gym.make("MiniHack-HideNSeek-Mapped-v0",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/project_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH


In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_N.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/project_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH
#REMOVE IMPOSSIBLE TO PERFORM ANY ACTION

In [ ]:
ENV = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_H.des",
              observation_keys=('screen_descriptions','message','pixel','blstats'))
evaluate(num_ep = 500, max_steps = 30, kb_path = 'kbs/project_kb.pl', env = ENV, speed = "fast", show = False)

#ADD GRAPH
#REMOVE IMPOSSIBLE TO PERFORM ANY ACTION